In [7]:
num_misses = 364
num_switches = 11
num_false_positives = 19008
num_objects = 12038

s = 1 - (num_misses + num_switches + num_false_positives) / num_objects
s

-0.6101511879049677

In [4]:
import numpy as np

obj_counts = np.asarray([12038, 5386, 12449])
weights = obj_counts / np.sum(obj_counts)
vals = np.asarray([-61, -22.5, -16.3])
np.sum(vals * weights)

-35.430713353195195